In [26]:
import pandas as pd

rto_2022 = pd.read_excel('../reports/rto_2022.xlsx')
rto_2023 = pd.read_excel('../reports/rto_2023.xlsx')
rto_2024 = pd.read_excel('../reports/rto_2024.xlsx')
rto_2025 = pd.read_excel('../reports/rto_2025.xlsx')

def merge_rto_data(df_main, df_rto_info):
    merged_df = df_main.merge(df_rto_info, on='RTO Code', how='left')

    cols = list(merged_df.columns)

    rto_index = cols.index('RTO Code')

    for col in ['District', 'City']:  # Reverse order to insert correctly
        if col in cols:
            cols.insert(rto_index + 1, cols.pop(cols.index(col)))

    return merged_df[cols]

### Separate missing RTO codes from RTO names

In [ ]:
import re
import pandas as pd

def extract_missing_rto_codes(df):
    # 1. Remove trailing date-like brackets: (05 Mar 2024)
    df['RTO'] = df['RTO'].str.replace(
        r'\(\s*\d{1,2} \w{3} \d{4}\s*\)$', '', regex=True
    ).str.strip()

    # 2. Replace multiple underscores and spaces with a single space for cleaner parsing
    df['RTO'] = df['RTO'].str.replace(r'[_\s]+', ' ', regex=True).str.strip()

    # 3. Mask for missing 'RTO Code'
    mask_missing = df['RTO Code'].isna() | df['RTO Code'].eq('')

    # 4. Extract 2-letter + 1-3 digit RTO code from end, e.g., HR1, TN33, KA70
    df.loc[mask_missing, 'RTO Code'] = (
        df.loc[mask_missing, 'RTO']
        .str.extract(r'([A-Za-z]{2}\d{1,3})$', expand=False)
        .str.upper()
    )

    return df


In [ ]:
rto_2023 = extract_missing_rto_codes(rto_2023)

In [ ]:
def missing_rto_code(df):
    faulty_rtos = df[df['RTO Code'].isna() | df['RTO Code'].eq('')]["RTO"].unique().tolist()
    return faulty_rtos

In [ ]:
missing_rto_code(rto_2025)

### Join RTO city mapping to main Dataframes

In [ ]:
rto_df = pd.read_excel('../reports/rto_code_mapping.xlsx')
missing_codes_mapping = pd.read_csv('../crazy_reports/missing_mapping.csv')

In [ ]:
combined_rto = pd.read_csv("../crazy_reports/combined_rto_mapping.csv")

In [ ]:
rto_2022_city = merge_rto_data(rto_2022, combined_rto)

rto_2022_city.head()

In [ ]:
rto_2023_city = merge_rto_data(rto_2023, combined_rto)

rto_2023_city.head()

In [ ]:
rto_2024_city = merge_rto_data(rto_2024, combined_rto)

rto_2024_city.head()

In [ ]:
rto_2025_city = merge_rto_data(rto_2025, combined_rto)

rto_2025_city.head()

In [ ]:
rto_2022_city.to_excel('../crazy_reports/rto_2022_city.xlsx')
rto_2023_city.to_excel('../crazy_reports/rto_2023_city.xlsx')
rto_2024_city.to_excel('../crazy_reports/rto_2024_city.xlsx')
rto_2025_city.to_excel('../crazy_reports/rto_2025_city.xlsx')

### Codes with missing city and district

In [ ]:
missing_rto_codes = rto_2023_city[
    rto_2023_city['City'].isna() | rto_2023_city['City'].eq('') |
    rto_2023_city['District'].isna() | rto_2023_city['District'].eq('')
]['RTO Code'].unique()

missing_rto_codes

In [ ]:
len(missing_rto_codes)

In [ ]:
five = 0
four = 0

three_digit_codes = []
two_digit_codes = []
one_digit_codes = []

for code in missing_rto_codes:
    code = str(code)
    if len(code) == 5:
        three_digit_codes.append(code)
        five = five + 1
    elif len(code) == 4:
        two_digit_codes.append(code)
        four = four + 1
    elif len(code) == 3:
        one_digit_codes.append(code)

print("Three Digit codes:", five)
print(three_digit_codes)
print("Two Digit codes:", four)
print(two_digit_codes)
print("Single Digit codes:", len(missing_rto_codes) - four - five)
print(one_digit_codes)


In [ ]:
rto_2025_city[rto_2025_city["RTO Code"] == "WB80"]

In [ ]:
universal_missing_codes = [
    'HP3', 'HP93', 'HP123', 'HP49', 'HP47', 'HP75', 'HP13',
    'HP30', 'HP42', 'HP76', 'HP10', 'WB80', 'AP302', 'AP602', 'HP11',
    'HP127', 'HP26', 'HP44', 'HP51', 'HP85', 'HP99', 'PB93', 'TN633',
    'AP702', 'HP77'
]

len(universal_missing_codes)

In [ ]:
combined_rto[combined_rto["RTO Code"].isin(universal_missing_codes)]

### function to update city and district for missing RTO codes

In [ ]:
def update_missing_rto_info(dataframes, missing_data_csv):
    """
    Updates missing City and District fields in multiple RTO DataFrames
    using the provided CSV containing RTO Code mappings.

    Parameters:
        dataframes (list of pd.DataFrame): List of RTO DataFrames to update.
        missing_data_csv (str): Path to CSV file containing RTO Code, City, District.
    
    Returns:
        list of pd.DataFrame: List of updated DataFrames.
    """
    # Load and prepare missing mapping data
    missing_data = pd.read_csv(missing_data_csv)
    missing_data.set_index('RTO Code', inplace=True)
    
    updated_dfs = []
    
    for df in dataframes:
        df = df.copy()
        df.set_index('RTO Code', inplace=True)
        df.update(missing_data[['City', 'District']])
        df.reset_index(inplace=True)
        updated_dfs.append(df)
    
    return updated_dfs

In [ ]:
import pandas as pd

missing_data = pd.read_csv('../crazy_reports/combined_rto_mapping.csv')
missing_data.set_index('RTO Code', inplace=True)

rto_dfs = [rto_2022_city, rto_2023_city, rto_2024_city, rto_2025_city]
rto_names = ['rto_2022_city', 'rto_2023_city', 'rto_2024_city', 'rto_2025_city']

# rto_dfs = [rto_2024_city]
# rto_names = ['rto_2024_city']

# Step 3: Loop through and apply updates
for i, df in enumerate(rto_dfs):    
    df.set_index('RTO Code', inplace=True)
    
    # Only update missing City or District
    df.update(rto_df[['City', 'District']])
    
    # Reset index to restore 'RTO Code' column
    df.reset_index(inplace=True)
    
    # Assign updated back to the original variable name (optional)
    globals()[rto_names[i]] = df  # if you want to rebind original names

    print(f"✅ Updated missing City/District for {rto_names[i]}")

### separating RTO codes from RTO names only for empty RTO codes.

In [ ]:
# Step 1: Remove only trailing date brackets
rto_2024_city['RTO'] = rto_2024_city['RTO'].str.replace(
    r'\(\s*\d{1,2} \w{3} \d{4}\s*\)$', '', regex=True
).str.strip()

# Step 2: Identify rows where RTO Code is missing
mask_missing_code = rto_2024_city['RTO Code'].isna() | rto_2024_city['RTO Code'].eq('')

# Step 3: Improved regex: match two letters + 1+ digits at the end
rto_2024_city.loc[mask_missing_code, 'RTO Code'] = (
    rto_2024_city.loc[mask_missing_code, 'RTO']
    .str.extract(r'([A-Za-z]{2}\d{1,3})$', expand=False)
    .str.upper()
)

In [ ]:
rto_2022_city[
    rto_2022_city["State"] == "Karnataka(68)"
]

### Correcting code in combined sheet

In [1]:
import pandas as pd

rto_2024 = pd.read_excel('../crazy_reports/rto_2024_city.xlsx')
rto_mapping = pd.read_csv('../crazy_reports/combined_rto_mapping.csv')

In [10]:
rto_names = []

for rto in rto_2024["RTO"].astype(str):
    parts = rto.strip().split()
    if len(parts) >= 2 and len(parts[0]) == 3:
        rto_names.append(parts[1])
    elif len(parts) >= 1:
        rto_names.append(parts[0])
rto_names = pd.Series(rto_names).unique().tolist()

rto_names

['Port',
 'Hindupur',
 'Tirupati',
 'Rajahmundry',
 'Narasaraopet',
 'Punganur',
 'Gudiwada',
 'Nandyal',
 'Chirala',
 'Anakapalli',
 'Parvathipuram',
 'Bhimavaram',
 'Regional',
 'Vijayawada',
 'Anantapur',
 'Guntakal',
 'Madanapalle',
 'Proddutur',
 'Amalapuram',
 'Bapatla',
 'Kurnool',
 'Machilipatnam',
 'Adoni',
 'Gudur',
 'Kandukur',
 'Palakonda',
 'Gajuwaka',
 'Jangareddygudem',
 'Nellore',
 'Prakasam',
 'Chittoor',
 'Srikakulam',
 'Kadiri',
 'Unit',
 'Vishakapatnam',
 'Nandigama',
 'Kavali',
 'Markapur',
 'Narsipatnam',
 'Kovvuru',
 'Vizianagaram',
 'Eluru',
 'Cuddapah',
 'Tadipatri',
 'Mandapeta',
 'Paderu',
 'Nuzvid',
 'Sullurpet',
 'Darsi',
 'Tadepalli',
 'Kakinada',
 'Rayachoti',
 'Vuyyuru',
 'Tanuku',
 'Dharamavaram',
 'Ramachandrapuram',
 'Piduguralla',
 'Jaggayyapet',
 'Palakole',
 'Guntur',
 'Ravulapalem',
 'Tenali',
 'Chintoor',
 'Itanagar',
 'Namsai',
 'Lower',
 'Upper',
 'Kamrup',
 'Karimganj',
 'Cachar',
 'Sonitpur',
 'Darrang',
 'Nalbari',
 'Barpeta',
 'Kokrajhar',


In [14]:
tamil_nadu = rto_2024[rto_2024["State"] == "Tamil Nadu"]

tamil_nadu_rto_names = []

for rto in tamil_nadu["RTO"].astype(str):
    parts = rto.strip().split()
    if len(parts) >= 2 and len(parts[0]) == 3:
        tamil_nadu_rto_names.append(parts[1])
    elif len(parts) >= 1:
        tamil_nadu_rto_names.append(parts[0])

tamil_nadu_rto_names = pd.Series(tamil_nadu_rto_names).unique().tolist()

tamil_nadu_rto_names

['Chennai',
 'Redhills',
 'Chengalpattu',
 'Tiruvallur',
 'Kancheepuram',
 'Meenambakkam',
 'Vellore',
 'Krishnagiri',
 'Tiruvannamalai',
 'Namakkal',
 'Dharmapuri',
 'Salem',
 'Cuddalore',
 'Viluppuram',
 'Erode',
 'Tiruchengode',
 'Gopichettipalayam',
 'Coimbatore',
 'Tiruppur',
 'Mettupalayam',
 'Pollachi',
 'Ooty',
 'Tiruchi',
 'Perambalur',
 'Karur',
 'Srirangam',
 'Thanjavur',
 'Tiruvarur',
 'Madurantagam',
 'Nagapattinam',
 'Poonamallee',
 'Sholinganallur',
 'Tambaram',
 'Gudiyatham',
 'Vaniyambadi',
 'Arani',
 'Paramathi',
 'Sankagiri',
 'Rasipuram',
 'Harur',
 'Omalure',
 'Chidambaram',
 'Pudukottai',
 'Virudhachalam',
 'Perundurai',
 'Neyveli',
 'Dindigul',
 'Tindivanam',
 'Ulundurpet',
 'Bhavani',
 'Sathyamangalam',
 'Madurai',
 'Avinashi',
 'Udumalpet',
 'Gudalore',
 'Tiruveranbur',
 'Manaparai',
 'Kulithali',
 'Thuraiyur',
 'Pattukottai',
 'Mannargudi',
 'Mayiladuthurai',
 'Mettur',
 'Attur',
 'Aranthangi',
 'Kangeyam',
 'Dharapuram',
 'Oddanchatram',
 'Batlagundu',
 'Pala

In [16]:
tn_correct_rto_codes = pd.DataFrame()

import numpy as np
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

# Prepare a list to collect matched rows
matched_rows = []

for rto_name in tamil_nadu_rto_names:
    best_match = None
    best_score = 0
    best_row = None
    for idx, row in rto_mapping.iterrows():
        city_name = str(row['City'])
        score = similar(rto_name, city_name)
        if score > best_score:
            best_score = score
            best_row = row
    if best_score >= 0.75:
        matched_rows.append({
            'RTO_Name': rto_name,
            'Matched_City': best_row['City'],
            'RTO_Code': best_row['RTO Code'],
            'District': best_row['District']
        })

tn_correct_rto_codes = pd.DataFrame(matched_rows)

tn_correct_rto_codes


,RTO_Name,Matched_City,RTO_Code,District
0,Redhills,Red Hills,TN18,Thiruvallur
1,Chengalpattu,Chengalpattu,TN19,Chengalpattu
2,Tiruvallur,Tiruvallur,TN20,Thiruvallur
3,Kancheepuram,Kanchipuram,TN21,Kanchipuram
4,Meenambakkam,Meenambakkam,TN22,Chennai
...,...,...,...,...
117,Ranipet,Ranipet,TN73,Ranipet
118,Nagercoil,Nagercoil,TN74,Kanniyakumari
119,Marthandam,Marthandam,TN75,Kanniyakumari
120,Tenkasi,Tenkasi,TN76,Tenkasi


In [17]:
correct_rto_codes = pd.DataFrame()

import numpy as np
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

# Prepare a list to collect matched rows
matched_rows = []

for rto_name in rto_names:
    best_match = None
    best_score = 0
    best_row = None
    for idx, row in rto_mapping.iterrows():
        city_name = str(row['City'])
        score = similar(rto_name, city_name)
        if score > best_score:
            best_score = score
            best_row = row
    if best_score >= 0.75:
        matched_rows.append({
            'RTO_Name': rto_name,
            'Matched_City': best_row['City'],
            'RTO_Code': best_row['RTO Code'],
            'District': best_row['District']
        })

correct_rto_codes = pd.DataFrame(matched_rows)

correct_rto_codes

,RTO_Name,Matched_City,RTO_Code,District
0,Hindupur,Hindupur,AP102,Anantapur
1,Tirupati,Tirupati,AP103,Chittoor
2,Rajahmundry,Rajahmundry,AP105,East Godavari
3,Narasaraopet,Narasaraopet,AP107,Palnadu
4,Punganur,Punganur,AP113,Chittoor
...,...,...,...,...
950,Udupi,Udupi,KA20,Udupi
951,Yadgiri,Yadgir,KA33,Yadgir
952,Yalahanka,Yelahanka,KA50,Bengaluru Urban
953,Sirsi,Sirsi,KA31,Uttara Kannada


### Changing 3 digit or wrong codes with correct

In [ ]:
changed_rows = []

# Make a copy to avoid modifying original
rto_2024_updated = rto_2024.copy()

for idx, row in correct_rto_codes.iterrows():
    matched_city = row['Matched_City']
    correct_code = row['RTO_Code']
    
    # Find all rows in rto_2024 with this city
    mask = rto_2024_updated['City'].str.lower() == matched_city.lower()
    # If there are no such rows, skip
    if not mask.any():
        continue
    
    # For those rows, check if RTO code is different from correct_code
    incorrect_mask = mask & (rto_2024_updated['RTO Code'] != correct_code)
    if incorrect_mask.any():
        # Record the changed rows before updating
        changed = rto_2024_updated.loc[incorrect_mask, ['State', 'City', 'RTO Code']].copy()
        changed['Correct_RTO_Code'] = correct_code
        changed_rows.append(changed)
        # Update the RTO code to the correct one
        rto_2024_updated.loc[incorrect_mask, 'RTO Code'] = correct_code

# Concatenate all changed rows into a single DataFrame
if changed_rows:
    changed_rto_codes = pd.concat(changed_rows, ignore_index=True)
else:
    changed_rto_codes = pd.DataFrame(columns=['State', 'City', 'RTO Code', 'Correct_RTO_Code'])

In [20]:
changed_rto_codes

,State,City,RTO Code,Correct_RTO_Code
0,Andhra Pradesh,Madanapalle,AP302,AP203
1,Andhra Pradesh,Madanapalle,AP302,AP203
2,Andhra Pradesh,Madanapalle,AP302,AP203
3,Andhra Pradesh,Madanapalle,AP302,AP203
4,Andhra Pradesh,Madanapalle,AP302,AP203
...,...,...,...,...
1044,Tamil Nadu,Manmangalam,TN637,TN47X
1045,Tamil Nadu,Manmangalam,TN637,TN47X
1046,Tamil Nadu,Manmangalam,TN637,TN47X
1047,Tamil Nadu,Manmangalam,TN637,TN47X


In [21]:
changed_rto_codes.groupby("State")["City"].nunique()

State
Andhra Pradesh       3
Bihar                1
Haryana              1
Himachal Pradesh    19
Madhya Pradesh       1
Meghalaya            1
Odisha               2
Punjab               2
Tamil Nadu          51
Uttar Pradesh        2
Name: City, dtype: int64

In [23]:
changed_rto_codes = changed_rto_codes.drop_duplicates()

changed_rto_codes

,State,City,RTO Code,Correct_RTO_Code
0,Andhra Pradesh,Madanapalle,AP302,AP203
6,Andhra Pradesh,Amalapuram,AP205,AP6
19,Andhra Pradesh,Amalapuram,AP602,AP6
20,Andhra Pradesh,Gajuwaka,AP231,AP33
40,Bihar,Aurangabad,BR26,MH20
...,...,...,...,...
983,Tamil Nadu,Thiruthuraipoondi,TN630,TN50Y
992,Tamil Nadu,Aravakurichi,TN632,TN47Y
1009,Tamil Nadu,Cheyyar,TN635,TN25Y
1029,Tamil Nadu,Manmangalam,TN637,TN47X


In [24]:
changed_rto_codes.to_excel('../crazy_reports/changed_rto_codes.xlsx', index=False)

In [28]:
rto_2022 = pd.read_excel('../crazy_reports/rto_2022_city.xlsx')
rto_2023 = pd.read_excel('../crazy_reports/rto_2023_city.xlsx')
rto_2024 = pd.read_excel('../crazy_reports/rto_2024_city.xlsx')
rto_2025 = pd.read_excel('../crazy_reports/rto_2025_city.xlsx')

def update_tn_rto_codes(df, correct_rto_codes):
    """
    Update RTO codes for Tamil Nadu only, based on correct_rto_codes DataFrame.
    Returns a tuple: (updated_df, changed_rto_codes)
    """
    changed_rows = []
    updated_df = df.copy()

    # Only consider rows in correct_rto_codes that are for Tamil Nadu
    tn_correct_rto_codes = correct_rto_codes[
        correct_rto_codes['District'].notnull()  # Defensive: only those with district info
    ]
    # If 'State' column exists in correct_rto_codes, filter for Tamil Nadu
    if 'State' in correct_rto_codes.columns:
        tn_correct_rto_codes = correct_rto_codes[correct_rto_codes['State'].str.lower() == 'tamil nadu']

    for idx, row in tn_correct_rto_codes.iterrows():
        matched_city = row['Matched_City']
        correct_code = row['RTO_Code']

        # Only update for Tamil Nadu rows
        mask_tn = updated_df['State'].str.lower() == 'tamil nadu'
        mask_city = updated_df['City'].str.lower() == matched_city.lower()
        mask = mask_tn & mask_city

        if not mask.any():
            continue

        incorrect_mask = mask & (updated_df['RTO Code'] != correct_code)
        if incorrect_mask.any():
            changed = updated_df.loc[incorrect_mask, ['State', 'City', 'RTO Code']].copy()
            changed['Correct_RTO_Code'] = correct_code
            changed_rows.append(changed)
            updated_df.loc[incorrect_mask, 'RTO Code'] = correct_code

    if changed_rows:
        changed_rto_codes = pd.concat(changed_rows, ignore_index=True)
    else:
        changed_rto_codes = pd.DataFrame(columns=['State', 'City', 'RTO Code', 'Correct_RTO_Code'])

    return updated_df, changed_rto_codes

rto_2024_updated, changed_rto_codes = update_tn_rto_codes(rto_2024, correct_rto_codes)
rto_2025_updated, changed_rto_codes = update_tn_rto_codes(rto_2025, correct_rto_codes)
rto_2022_updated, changed_rto_codes = update_tn_rto_codes(rto_2022, correct_rto_codes)
rto_2023_updated, changed_rto_codes = update_tn_rto_codes(rto_2023, correct_rto_codes)

In [30]:
rto_2022_updated.to_excel('../final_reports/rto_2022_city_updated.xlsx', index=False)
rto_2023_updated.to_excel('../final_reports/rto_2023_city_updated.xlsx', index=False)
rto_2024_updated.to_excel('../final_reports/rto_2024_city_updated.xlsx', index=False)
rto_2025_updated.to_excel('../final_reports/rto_2025_city_updated.xlsx', index=False)

In [31]:
with pd.ExcelWriter('../final_reports/ev_sales_data_combined.xlsx', engine='openpyxl') as writer:
    rto_2022_updated.to_excel(writer, sheet_name='2022', index=False)
    rto_2023_updated.to_excel(writer, sheet_name='2023', index=False)
    rto_2024_updated.to_excel(writer, sheet_name='2024', index=False)
    rto_2025_updated.to_excel(writer, sheet_name='2025', index=False)

    # Create a master sheet with data for all years
    master_df = pd.concat(
        [
            rto_2022_updated.assign(Year=2022),
            rto_2023_updated.assign(Year=2023),
            rto_2024_updated.assign(Year=2024),
            rto_2025_updated.assign(Year=2025)
        ],
        ignore_index=True
    )
    master_df.to_excel(writer, sheet_name='master', index=False)